# **Домашнее задание №5**

1. Напишите код, который будет искать решение для задачи «Ханойская башня». Требуется найти решение при помощи уравнения Беллмана и алгоритма Дейкстры.

2. Напишите код, который будет искать решение для задачи «Сокобан» с одним ящиком.

3. «Со звездочкой» Напишите код, который будет искать решение для задачи «Сокобан» для произвольного числа ящиков.



Решения, не использующие уравнение Беллмана или алгоритм Дейкстры, будут оцениваться в 0 баллов!

Для 2 и 3 задания в файлах приведены примерные входные данные:

X - финальная клетка,

O - ящик,

Y - рабочий,

'#' - стена.

# **Библиотеки**

In [24]:
import heapq as hq  
from numpy import inf  
import heapq as hq  
from collections import defaultdict  
from collections import deque

# **Задание_1**

Код решает Ханойскую башню для n дисков с помощью Дейкстры, моделируя состояния как кортежи трёх стержней и генерируя соседей в next_nodes через валидные перемещения (меньший диск на больший или пустой), алгоритм dijkstra применяет приоритетную очередь для поиска кратчайшего пути (вес ходов=1) от старта к финишу, сохраняя список состояний

In [25]:
# Генерирует все возможные следующие состояния (соседние вершины в графе)
# для текущего состояния Ханойской башни, проверяя валидные перемещения дисков между стержнями
def next_nodes(current_state):
    poles = [list(p) for p in current_state]  
    neighbors = []
    for from_pole in range(3):
        if not poles[from_pole]:
            continue
        disk = poles[from_pole][-1]  
        for to_pole in range(3):
            if from_pole == to_pole:
                continue
            if not poles[to_pole] or disk < poles[to_pole][-1]:  
                new_poles = [p[:] for p in poles]  
                new_poles[from_pole].pop()  
                new_poles[to_pole].append(disk)  
                new_state = tuple(tuple(sorted(p, reverse=True)) for p in new_poles)
                neighbors.append(new_state)
    return neighbors

# Функция реализует алгоритм Дейкстры
def dijkstra(n_disks):
    beg = (tuple(range(n_disks, 0, -1)), (), ())  
    fin = ((), (), tuple(range(n_disks, 0, -1)))  
    QUEUE = []
    DIST = {}
    PATH = {}

    hq.heappush(QUEUE, (0, beg))  
    DIST[beg] = 0
    PATH[beg] = [beg]  

    while QUEUE:
        dist, cur = hq.heappop(QUEUE)
        if cur == fin:  
            return PATH[cur], dist  
        next_step = next_nodes(cur)
        for node in next_step:
            new_dist = dist + 1  
            if (node not in DIST) or (new_dist < DIST[node]):
                DIST[node] = new_dist
                PATH[node] = PATH[cur] + [node]
                hq.heappush(QUEUE, (new_dist, node))
    return None, None

# Краш тест всей этой страшной конструкции 
n = 3 # Диски для теста 
path, num_moves = dijkstra(n)
if path:
    print(f"Количество ходов: {num_moves} (для 3 дисков оптимально 7, вдруг кто-то не знал)")
    print("Ходы и состояния (визуал):")
    for state in path:
        print(state)
else:
    print("Решение не найдено, увы")

Количество ходов: 7 (для 3 дисков оптимально 7, вдруг кто-то не знал)
Ходы и состояния (визуал):
((3, 2, 1), (), ())
((3, 2), (), (1,))
((3,), (2,), (1,))
((3,), (2, 1), ())
((), (2, 1), (3,))
((1,), (2,), (3,))
((1,), (), (3, 2))
((), (), (3, 2, 1))


Начальное состояние: ((3, 2, 1), (), ()), где все диски на A (3 внизу, затем 2, сверху 1) - объясняю супер 

((3, 2), (), (1,)) — Переместили 1 с A на C

((3,), (2,), (1,)) — Переместили 2 с A на B

((3,), (2, 1), ()) — Переместили 1 с C на B

((), (2, 1), (3,)) — Переместили 3 с A на C

((1,), (2,), (3,)) — Переместили 1 с B на A 

((1,), (), (3, 2)) — Переместили 2 с B на C

Конечное состояние: ((), (), (3, 2, 1)) — Все диски на C 


# **Задание 2** (Обязательно посмотреть видосик из файла)

Этот код реализует решение задачи Сокобана с одним ящиком с помощью алгоритма BFS, где состояния представлены как кортежи позиций рабочего и ящика, а ходы включают движение или толчок в четырёх направлениях, код парсит карту из файла, находит начальные позиции, строит путь до цели и реконструирует последовательность шагов, выводя начальную и финальную конфигурации сетки для визуализации

In [26]:
# функция читает файл с картой Сокобана и преобразует его в двумерный список
def parse_sokoban_map(filename):
    with open(filename, 'r') as f:
        lines = [line.rstrip('\n') for line in f.readlines()]
    max_len = max(len(line) for line in lines)
    grid = [list(line.ljust(max_len, ' ')) for line in lines]
    return grid

# Эта функция ищет позиции ключевых элементов на сетке
def find_positions(grid):
    worker = box = target = None # инициализация 
    for i, row in enumerate(grid): # перебираем строки сетки
        for j, cell in enumerate(row):
            if cell == 'Y': # рабочий 
                worker = (i, j)
            elif cell == 'O': # ящик
                box = (i, j)
            elif cell == 'X': # таргет или цель или гол сколько разных слов...
                target = (i, j)
    return worker, box, target, grid

# Это основная функция для решения задачи с помощью BFS (сердечко кода)
# Она моделирует состояния как (позиция рабочего + позиция ящика) и ищет путь 
def solve_sokoban_one_box(filename):
    worker, box, target, grid = find_positions(parse_sokoban_map(filename))
    rows = len(grid)
    cols = max(len(row) for row in grid) if grid else 0
    
    start = (*worker, *box) # cоздаёт начальное состояние как кортеж
    goal_box = target # я все-таки обозвал как гол:) можно уже дальше не писать:)
    
    dirs = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    dir_names = ['right', 'down', 'left', 'up']
    
    ################ ОБЯЗАТЕЛЬНО ПРОЧИТАТЬ ########################################
    # При написании этого кусочка в коде у меня в голове визуализировалось это ####
    #             https://youtu.be/dncVUXs_OP8?si=SdwOyoJE12msdr8u&t=84           #
    
    # Думаю, вы улыбнетесь, я улыбнулся и мне было забавно                        # 
    ###############################################################################
    
    # проверяет, можно ли двигаться в клетку
    def can_move_to(x, y):
        if not (0 <= x < rows and 0 <= y < len(grid[x])): # Проверяет границы сетки
            return False
        return grid[x][y] != '#'
    
    queue = deque([start]) # Создаёт очередь для BFS
    dist = {start: 0} # Словарь расстояний
    parent = {} # Словарь родителей для реконструкции пути
    moves = {} # Словарь для хранения названий ходов
    
    visited = set() # Множество посещенных состояний
    
    while queue: # Цикл BFS
        state = queue.popleft() # Извлекает состояние из начала очереди
        wx, wy, bx, by = state
        
        if (bx, by) == goal_box:
            path = []
            move_path = []
            cur = state
            while cur in parent:
                path.append(cur)
                move_path.append(moves.get(cur, ''))
                cur = parent[cur]
            path.append(start)
            path.reverse()
            move_path.reverse()
            return path, move_path[1:], grid, target 
        
        if state in visited: # Если состояние уже посещено
            continue
        visited.add(state)
        
        # И дальше 5000 случаев, я умру их расписывать 
        for i, (dx, dy) in enumerate(dirs):
            nx, ny = wx + dx, wy + dy
            if not can_move_to(nx, ny):
                continue
            if (nx, ny) != (bx, by):
                new_state = (nx, ny, bx, by)
                move_name = 'move ' + dir_names[i]
            else:
                nbx, nby = bx + dx, by + dy
                if not can_move_to(nbx, nby):
                    continue
                new_state = (nx, ny, nbx, nby)
                move_name = 'push ' + dir_names[i]
            
            if new_state not in dist:
                dist[new_state] = dist[state] + 1
                parent[new_state] = state
                moves[new_state] = move_name
                queue.append(new_state)
    
    return None, None, None, None

# Функция для печати текущего состояния сетки (уважаю визуал)
def print_grid(grid, worker, box, target):
    display_grid = [row[:] for row in grid]
    
    # Убираем оригинальные Y, O, X для чистоты, но сохраним X если box не на нем
    for i in range(len(display_grid)):
        for j in range(len(display_grid[i])):
            if display_grid[i][j] in ['Y', 'O', 'X']:
                display_grid[i][j] = ' '
    # Ставим worker, box - бууууум
    wx, wy = worker
    bx, by = box
    display_grid[wx][wy] = 'Y'
    if (bx, by) == target:
        display_grid[bx][by] = '*'  
    else:
        display_grid[bx][by] = 'O'
        display_grid[target[0]][target[1]] = 'X'
    # Print
    for row in display_grid:
        print(''.join(row))
    print()  # Пустая строка для разделения

# основной блок для запуска скрипта как программы
if __name__ == "__main__":
    path, move_path, grid, target = solve_sokoban_one_box('sokoban.txt')
    if path:
        print("Решение найдено! Количество шагов:", len(move_path))
        
        # Показываем начальную конфигурацию ("Было")
        initial_state = path[0]
        initial_worker = initial_state[:2]
        initial_box = initial_state[2:]
        print("Было (начальная конфигурация):")
        print_grid(grid, initial_worker, initial_box, target)
        
        # Показываем финальную конфигурацию ("Стало")
        final_state = path[-1]
        final_worker = final_state[:2]
        final_box = final_state[2:]
        print("Стало (финальная конфигурация):")
        print_grid(grid, final_worker, final_box, target)
    else:
        print("Нет решения")

Решение найдено! Количество шагов: 30
Было (начальная конфигурация):
####### 
#X#   # 
# # #O##
#      #
# # ## #
#  Y   #
########

Стало (финальная конфигурация):
####### 
#*#   # 
#Y# # ##
#      #
# # ## #
#      #
########



Звёздочка * в выводе конфигурации карты обозначает ящик, который находится на финальной клетке 

# **Задание 3***

Этот код решает задачу Сокобана с любым числом ящиков, используя алгоритм Дейкстры для поиска кратчайшего пути: те парсит карту из файлика, определяет позиции рабочего, ящиков и целей, моделирует состояния как комбинацию позиций и генерирует ходы (движение или толчок), в чем прикол, приоритетная очередь обеспечивает оптимальность, релаксируя расстояния по уравнению Беллмана с весом ребер 1, и останавливается, когда все ящики на целях, а далее в конце реконструирует путь, выводит начальную/финальную конфигурации и последовательность ходов для тестового файла

In [27]:
# функция читает файл с картой Сокобана и преобразует его в двумерный список
def parse_sokoban_map(filename):
    with open(filename, 'r') as f:
        lines = [line.rstrip('\n') for line in f.readlines()]
    max_len = max(len(line) for line in lines)
    grid = [list(line.ljust(max_len, ' ')) for line in lines]
    return grid

# Эта функция ищет позиции рабочего, ящиков (как множество) и целей (как множество)
def find_positions(grid):
    worker = None
    boxes = set()
    goals = set()
    for i, row in enumerate(grid):
        for j, cell in enumerate(row):
            if cell == 'Y':
                worker = (i, j)
            elif cell == 'O':
                boxes.add((i, j))
            elif cell == 'X':
                goals.add((i, j))
    return worker, boxes, goals, grid

# Основная функция для решения с помощью Дейкстры 
# (приоритетная очередь heapq для кратчайшего пути с весами ребер = 1, что соответствует уравнению Беллмана)

def solve_sokoban(filename):
    worker, boxes, goals, grid = find_positions(parse_sokoban_map(filename))
    rows = len(grid)
    cols = max(len(row) for row in grid) if grid else 0
    start = (worker, frozenset(boxes))  # State: (worker_pos, frozenset(box_pos))
    goal_set = goals  # Set of goal positions
    
    dirs = [(-1, 0), (0, -1), (1, 0), (0, 1)]
    dir_names = ['up', 'left', 'down', 'right']
    
    def can_move_to(x, y):
        if not (0 <= x < rows and 0 <= y < len(grid[x])):
            return False
        return grid[x][y] != '#'
    
    queue = [] # Пустой список для приоритетной очереди
    hq.heappush(queue, (0, start))  
    dist = defaultdict(lambda: float('inf'))
    dist[start] = 0
    parent = {}
    moves = {}
    visited = set()
    
    while queue:
        current_dist, state = hq.heappop(queue)
        worker_pos, current_boxes = state
        if current_dist > dist[state]:
            continue
        
        if set(current_boxes) == goal_set:
            path = []
            move_path = []
            cur = state
            while cur in parent:
                path.append(cur)
                move_path.append(moves.get(cur, ''))
                cur = parent[cur]
            path.append(start)
            path.reverse()
            move_path.reverse()
            return path, move_path, grid, goals
        
        if state in visited:
            continue
        visited.add(state)
        
        wx, wy = worker_pos
        for i, (dx, dy) in enumerate(dirs):
            nx, ny = wx + dx, wy + dy
            if not can_move_to(nx, ny):
                continue
            box_pos = (nx, ny)
            if box_pos not in current_boxes:
                new_worker = (nx, ny)
                new_boxes = current_boxes
                move_name = 'move ' + dir_names[i]
            else:
                nbx, nby = nx + dx, ny + dy
                if not can_move_to(nbx, nby) or (nbx, nby) in current_boxes:
                    continue
                new_worker = (nx, ny)
                new_boxes = (current_boxes - {box_pos}) | {(nbx, nby)}
                move_name = 'push ' + dir_names[i]
            
            new_state = (new_worker, frozenset(new_boxes))
            new_dist = current_dist + 1  # Edge weight 1, as in Bellman relaxation
            if new_dist < dist[new_state]:
                dist[new_state] = new_dist
                parent[new_state] = state
                moves[new_state] = move_name
                hq.heappush(queue, (new_dist, new_state))
    
    return None, None, None, None

# Визуализация 
def print_grid(grid, worker, boxes, goals):
    display_grid = [row[:] for row in grid]
    for i in range(len(display_grid)):
        for j in range(len(display_grid[i])):
            if display_grid[i][j] in ['Y', 'O', 'X']:
                display_grid[i][j] = ' '
    wx, wy = worker
    display_grid[wx][wy] = 'Y'
    for bx, by in boxes:
        if (bx, by) in goals:
            display_grid[bx][by] = '*'  
        else:
            display_grid[bx][by] = 'O'
    for gx, gy in goals - set(boxes):
        display_grid[gx][gy] = 'X'  
    # Print
    for row in display_grid:
        print(''.join(row))
    print()  

# Тест 
if __name__ == "__main__":
    file = 'sokoban2.txt'  
    print(f"Хм... файлик ням-ням {file}...")
    path, move_path, grid, goals = solve_sokoban(file)
    if path:
        print("Решение найдено! Всем миска риса и кошка жена! Количество шагов:", len(move_path))
        initial_state = path[0]
        initial_worker, initial_boxes = initial_state
        print("Было (конфигурация из файлика):")
        print_grid(grid, initial_worker, initial_boxes, goals)

        final_state = path[-1]
        final_worker, final_boxes = final_state
        print("Стало (Стало после наворотов):")
        print_grid(grid, final_worker, final_boxes, goals)
        print("Шаги:", ' -> '.join(move_path))
    else:
        print("Нет решения")

Хм... файлик ням-ням sokoban2.txt...
Решение найдено! Всем миска риса и кошка жена! Количество шагов: 48
Было (конфигурация из файлика):
####### 
#X#   # 
#X# #O##
#    O #
# # ## #
#  Y   #
########

Стало (Стало после наворотов):
####### 
#*#   # 
#*# # ##
#Y     #
# # ## #
#      #
########

Шаги: move right -> move right -> move right -> move up -> move up -> push left -> push left -> push left -> push left -> move right -> move down -> move down -> move left -> move left -> move up -> push up -> push up -> move down -> move right -> move right -> move up -> move up -> move right -> move right -> push down -> move up -> move left -> move left -> move down -> move down -> move down -> move down -> move right -> move right -> move right -> move up -> move up -> push left -> push left -> push left -> push left -> move right -> move down -> move down -> move left -> move left -> move up -> push up
